In [1]:
import sqlite3
import pandas as pd
import numpy as np
conn = sqlite3.connect('high_freq.db')
df = pd.read_sql("""select z.time,
    z.thscode,
    z.open,
    z.high,
    z.low,
    z.close,
    z.avgPrice,
    z.volume,
    z.amount,
    z.change,
    z.changeRatio,
    z.turnoverRatio,
    z.sellVolume,
    z.buyVolume, s.Security_name from zhangtinggu z join Security_name s on z.thscode = s.thscode
    where (z.thscode like '0%' 
    or z.thscode like '603%' 
    or z.thscode like '600%'
     or z.thscode like '601%')
     and s.Security_name not like '%ST%'""",conn)
conn.close()
print(df)



                     time    thscode  open  high   low  close  avgPrice  \
0        2023-11-01 09:30  600100.SH  8.26  8.26  8.26   8.26  8.260000   
1        2023-11-01 09:31  600100.SH  8.25  8.28  8.21   8.23  8.247400   
2        2023-11-01 09:32  600100.SH  8.21  8.34  8.21   8.33  8.276600   
3        2023-11-01 09:33  600100.SH  8.32  8.36  8.32   8.35  8.331000   
4        2023-11-01 09:34  600100.SH  8.35  8.42  8.35   8.41  8.369300   
...                   ...        ...   ...   ...   ...    ...       ...   
5092566  2023-09-19 14:56  002173.SZ  8.09  8.10  8.09   8.09  8.096483   
5092567  2023-09-19 14:57  002173.SZ  8.10  8.10  8.09   8.09  8.098415   
5092568  2023-09-19 14:58  002173.SZ  8.10  8.10  8.10   8.10  8.100000   
5092569  2023-09-19 14:59  002173.SZ  8.10  8.10  8.10   8.10       NaN   
5092570  2023-09-19 15:00  002173.SZ  8.09  8.09  8.09   8.09  8.090000   

            volume      amount  change  changeRatio  turnoverRatio  \
0         841700.0   6952442.

In [2]:
df = df.drop_duplicates(subset=['time', 'thscode'], keep='first')


df = df.drop_duplicates(subset=['time', 'thscode'], keep='first')
df['time'] = pd.to_datetime(df['time'])
# 然后从'time'列提取日期，并创建新的'date'列
df['date'] = df['time'].dt.date


print(df)

                       time    thscode  open  high   low  close  avgPrice  \
0       2023-11-01 09:30:00  600100.SH  8.26  8.26  8.26   8.26  8.260000   
1       2023-11-01 09:31:00  600100.SH  8.25  8.28  8.21   8.23  8.247400   
2       2023-11-01 09:32:00  600100.SH  8.21  8.34  8.21   8.33  8.276600   
3       2023-11-01 09:33:00  600100.SH  8.32  8.36  8.32   8.35  8.331000   
4       2023-11-01 09:34:00  600100.SH  8.35  8.42  8.35   8.41  8.369300   
...                     ...        ...   ...   ...   ...    ...       ...   
5092325 2023-09-19 14:56:00  002173.SZ  8.09  8.10  8.09   8.09  8.096483   
5092326 2023-09-19 14:57:00  002173.SZ  8.10  8.10  8.09   8.09  8.098415   
5092327 2023-09-19 14:58:00  002173.SZ  8.10  8.10  8.10   8.10  8.100000   
5092328 2023-09-19 14:59:00  002173.SZ  8.10  8.10  8.10   8.10       NaN   
5092329 2023-09-19 15:00:00  002173.SZ  8.09  8.09  8.09   8.09  8.090000   

            volume      amount  change  changeRatio  turnoverRatio  \
0    

In [5]:
import numpy as np

#峰度和偏度#


# 确保DataFrame是按照thscode和date排序的
df = df.sort_values(by=['thscode', 'date'])


# 计算对数收益率，但首先排除每天的第一条记录
df['log_return'] = np.where(
    df['time'].dt.normalize() != df['time'].shift(1).dt.normalize(), 
    np.nan,  # 如果是新的一天，则不计算对数收益率，赋值为NaN
    np.log(df['close'] / df['close'].shift(1))  # 否则，正常计算对数收益率
)

# 通过groupby计算每日的偏度和峰度
daily_skewness = df.groupby(['thscode', 'date'])['log_return'].skew().rename('daily_skewness')
daily_kurtosis = df.groupby(['thscode', 'date'])['log_return'].apply(pd.Series.kurt).rename('daily_kurtosis')
daily_variance = df.groupby(['thscode', 'date'])['log_return'].var().rename('daily_variance')

daily_skewness = daily_skewness.reset_index()
daily_kurtosis = daily_kurtosis.reset_index()
daily_variance = daily_variance.reset_index()

# 将计算结果合并回原始DataFrame
df = df.merge(daily_skewness, on=['thscode', 'date'], how='left')
df = df.merge(daily_kurtosis, on=['thscode', 'date'], how='left')
df = df.merge(daily_variance, on=['thscode', 'date'], how='left')

print(df)

                       time    thscode   open   high    low  close   avgPrice  \
0       2023-08-21 09:30:00  000004.SZ  16.64  16.64  16.64  16.64  16.640000   
1       2023-08-21 09:31:00  000004.SZ  16.62  16.85  16.52  16.85  16.672619   
2       2023-08-21 09:32:00  000004.SZ  16.88  16.94  16.80  16.80  16.822798   
3       2023-08-21 09:33:00  000004.SZ  16.81  16.95  16.80  16.93  16.896727   
4       2023-08-21 09:34:00  000004.SZ  16.93  17.03  16.90  17.03  16.949838   
...                     ...        ...    ...    ...    ...    ...        ...   
2553149 2023-11-01 14:56:00  603982.SH  16.68  16.68  16.68  16.68  16.680000   
2553150 2023-11-01 14:57:00  603982.SH  16.68  16.68  16.68  16.68  16.680000   
2553151 2023-11-01 14:58:00  603982.SH  16.68  16.68  16.68  16.68        NaN   
2553152 2023-11-01 14:59:00  603982.SH  16.68  16.68  16.68  16.68        NaN   
2553153 2023-11-01 15:00:00  603982.SH  16.68  16.68  16.68  16.68  16.680000   

           volume     amoun

In [6]:
# 定义计算Pearson相关系数的函数
def pearson_correlation(group):
    correlation = group['close'].corr(group['volume'])
    group['pearson_corr'] = correlation
    return group

# 按股票代码和日期分组并应用函数，不添加分组键到索引
df = df.groupby(['thscode', 'date']).apply(pearson_correlation)

# 重置索引，如果你有多级索引的话
df = df.reset_index(drop=True)

print(df)

                       time    thscode   open   high    low  close   avgPrice  \
0       2023-08-21 09:30:00  000004.SZ  16.64  16.64  16.64  16.64  16.640000   
1       2023-08-21 09:31:00  000004.SZ  16.62  16.85  16.52  16.85  16.672619   
2       2023-08-21 09:32:00  000004.SZ  16.88  16.94  16.80  16.80  16.822798   
3       2023-08-21 09:33:00  000004.SZ  16.81  16.95  16.80  16.93  16.896727   
4       2023-08-21 09:34:00  000004.SZ  16.93  17.03  16.90  17.03  16.949838   
...                     ...        ...    ...    ...    ...    ...        ...   
2553149 2023-11-01 14:56:00  603982.SH  16.68  16.68  16.68  16.68  16.680000   
2553150 2023-11-01 14:57:00  603982.SH  16.68  16.68  16.68  16.68  16.680000   
2553151 2023-11-01 14:58:00  603982.SH  16.68  16.68  16.68  16.68        NaN   
2553152 2023-11-01 14:59:00  603982.SH  16.68  16.68  16.68  16.68        NaN   
2553153 2023-11-01 15:00:00  603982.SH  16.68  16.68  16.68  16.68  16.680000   

           volume     amoun

In [7]:
#成交量分布因子


import pandas as pd

# 确保 'time' 列是日期时间格式
df['time'] = pd.to_datetime(df['time'])

# 创建时间段边界
morning_times = pd.date_range(start='09:30:00', end='11:30:00', periods=5).time
afternoon_times = pd.date_range(start='13:00:00', end='15:00:00', periods=5).time

# 分配时间段
def assign_time_bin(row_time):
    for i in range(4):  # 注意这里的范围是4，因为我们有5个时间点，形成4个时间段
        if morning_times[i] <= row_time <= morning_times[i+1]:
            return 'M' + str(i+1)
        elif afternoon_times[i] <= row_time <= afternoon_times[i+1]:
            return 'A' + str(i+1)
    return 'Out_of_session'

# 应用时间段分配
df['time_bin'] = df['time'].dt.time.apply(assign_time_bin)

# 定义函数来计算每只股票每天的成交量分布因子
def calculate_volume_distribution_factor(group):
    volume_by_bin = group.groupby('time_bin')['volume'].sum()
    total_volume = group['volume'].sum()
    volume_distribution_factor = volume_by_bin / total_volume
    # 映射分布因子回group
    group['volume_distribution_factor'] = group['time_bin'].map(volume_distribution_factor)
    return group

# 对每只股票每天的数据应用函数并合并结果
df = df.groupby(['thscode', 'date']).apply(calculate_volume_distribution_factor).reset_index(drop=True)


print(df)


                       time    thscode   open   high    low  close   avgPrice  \
0       2023-08-21 09:30:00  000004.SZ  16.64  16.64  16.64  16.64  16.640000   
1       2023-08-21 09:31:00  000004.SZ  16.62  16.85  16.52  16.85  16.672619   
2       2023-08-21 09:32:00  000004.SZ  16.88  16.94  16.80  16.80  16.822798   
3       2023-08-21 09:33:00  000004.SZ  16.81  16.95  16.80  16.93  16.896727   
4       2023-08-21 09:34:00  000004.SZ  16.93  17.03  16.90  17.03  16.949838   
...                     ...        ...    ...    ...    ...    ...        ...   
2553149 2023-11-01 14:56:00  603982.SH  16.68  16.68  16.68  16.68  16.680000   
2553150 2023-11-01 14:57:00  603982.SH  16.68  16.68  16.68  16.68  16.680000   
2553151 2023-11-01 14:58:00  603982.SH  16.68  16.68  16.68  16.68        NaN   
2553152 2023-11-01 14:59:00  603982.SH  16.68  16.68  16.68  16.68        NaN   
2553153 2023-11-01 15:00:00  603982.SH  16.68  16.68  16.68  16.68  16.680000   

           volume     amoun

In [8]:
import pandas as pd
import numpy as np


# 计算每只股票每天的成交量变化量
df['volume_change'] = df.groupby(['thscode', 'date'])['volume'].diff().fillna(0)

# 定义指示函数
def indicator(x, direction):
    if direction == 'up':
        return x if x > 0 else 0
    elif direction == 'down':
        return x if x < 0 else 0
    else:
        return 0

# 应用指示函数并计算平方
df['vol_up_squared'] = df['volume_change'].apply(lambda x: indicator(x, 'up')) ** 2
df['vol_down_squared'] = df['volume_change'].apply(lambda x: indicator(x, 'down')) ** 2

# 计算每只股票每天的指标
def calculate_daily_metrics(group):
    volume_upward_volatility = np.sqrt(group['vol_up_squared'].sum())
    volume_downward_volatility = np.sqrt(group['vol_down_squared'].sum())
    total_vol_squared = group['volume_change'].pow(2).sum()
    volume_upward_volatility_ratio = volume_upward_volatility / np.sqrt(total_vol_squared)
    volume_downward_volatility_ratio = volume_downward_volatility / np.sqrt(total_vol_squared)
    
    # 返回一个包含所有计算指标的Series
    return pd.Series({
        'volume_upward_volatility': volume_upward_volatility,
        'volume_downward_volatility': volume_downward_volatility,
        'volume_upward_volatility_ratio': volume_upward_volatility_ratio,
        'volume_downward_volatility_ratio': volume_downward_volatility_ratio
    })

# 应用计算函数并创建一个新的DataFrame
metrics_df = df.groupby(['thscode', 'date']).apply(calculate_daily_metrics).reset_index()

# 将计算结果合并回原始DataFrame
df = df.merge(metrics_df, on=['thscode', 'date'], how='left')

print(df.head())


                 time    thscode   open   high    low  close   avgPrice  \
0 2023-08-21 09:30:00  000004.SZ  16.64  16.64  16.64  16.64  16.640000   
1 2023-08-21 09:31:00  000004.SZ  16.62  16.85  16.52  16.85  16.672619   
2 2023-08-21 09:32:00  000004.SZ  16.88  16.94  16.80  16.80  16.822798   
3 2023-08-21 09:33:00  000004.SZ  16.81  16.95  16.80  16.93  16.896727   
4 2023-08-21 09:34:00  000004.SZ  16.93  17.03  16.90  17.03  16.949838   

     volume     amount  change  ...  pearson_corr  time_bin  \
0  164200.0  2732288.0   -0.01  ...      0.294682        M1   
1  291700.0  4863403.0    0.21  ...      0.294682        M1   
2  339500.0  5711340.0   -0.05  ...      0.294682        M1   
3  299400.0  5058880.0    0.13  ...      0.294682        M1   
4  278400.0  4718835.0    0.10  ...      0.294682        M1   

   volume_distribution_factor  volume_change vol_up_squared vol_down_squared  \
0                      0.4938            0.0   0.000000e+00     0.000000e+00   
1         

In [9]:

# 首先，计算每只股票每天的成交量变化量
df['volume_change'] = df.groupby(['thscode', 'date'])['volume'].diff().fillna(0)

# 接下来，计算每只股票每天的成交量上行波动性和下行波动性
df['vol_up_squared'] = np.where(df['volume_change'] > 0, df['volume_change']**2, 0)
df['vol_down_squared'] = np.where(df['volume_change'] < 0, df['volume_change']**2, 0)

# 每只股票每天的波动性指标
volatility_metrics = df.groupby(['thscode', 'date']).agg(
    volume_upward_volatility=pd.NamedAgg(column='vol_up_squared', aggfunc=lambda x: np.sqrt(x.sum())),
    volume_downward_volatility=pd.NamedAgg(column='vol_down_squared', aggfunc=lambda x: np.sqrt(x.sum()))
)

# 计算成交量上行和下行波动性比
total_vol_squared = df.groupby(['thscode', 'date'])['volume_change'].apply(lambda x: (x**2).sum())
volatility_metrics['volume_upward_volatility_ratio'] = volatility_metrics['volume_upward_volatility'] / np.sqrt(total_vol_squared)
volatility_metrics['volume_downward_volatility_ratio'] = volatility_metrics['volume_downward_volatility'] / np.sqrt(total_vol_squared)

# 重新创建result DataFrame
result = df.groupby(['thscode', 'date']).agg(
    daily_skewness=('daily_skewness', 'mean'),
    daily_kurtosis=('daily_kurtosis', 'mean'),
    daily_variance=('daily_variance', 'mean'),
    pearson_corr=('pearson_corr', 'mean')
).reset_index()

# 合并波动性指标
result = result.merge(volatility_metrics, on=['thscode', 'date'], how='left')


print(result)


         thscode        date  daily_skewness  daily_kurtosis  daily_variance  \
0      000004.SZ  2023-08-21        1.632163        8.968361        0.000005   
1      000004.SZ  2023-08-22        1.547498        8.702269        0.000004   
2      000004.SZ  2023-08-23        4.197470       45.139436        0.000015   
3      000004.SZ  2023-08-24        1.445023        7.397598        0.000010   
4      000004.SZ  2023-08-25       -1.297136       23.930925        0.000012   
...          ...         ...             ...             ...             ...   
10589  603982.SH  2023-10-26        2.373152       25.816678        0.000012   
10590  603982.SH  2023-10-27        1.253776        8.945336        0.000007   
10591  603982.SH  2023-10-30       -0.102672       11.926577        0.000007   
10592  603982.SH  2023-10-31       -0.345534        5.892632        0.000003   
10593  603982.SH  2023-11-01        3.493311       26.945392        0.000012   

       pearson_corr  volume_upward_vola

In [10]:

# 使用 pivot_table 将 'volume_distribution_factor' 分布按 'time_bin' 重塑成多列
time_bin_vdf = df.pivot_table(index=['thscode', 'date'], 
                              columns='time_bin', 
                              values='volume_distribution_factor', 
                              aggfunc='mean').reset_index()

# 将列索引简化，用 'time_bin' 的值作为新列名的前缀
time_bin_vdf.columns = [str(col) if col == 'thscode' or col == 'date' else f'time_bin_{col}' for col in time_bin_vdf.columns]

# 将这些 'time_bin' 列合并到 result DataFrame 中
result = pd.merge(result, time_bin_vdf, on=['thscode', 'date'], how='left')


result

,thscode,date,daily_skewness,daily_kurtosis,daily_variance,pearson_corr,volume_upward_volatility,volume_downward_volatility,volume_upward_volatility_ratio,volume_downward_volatility_ratio,time_bin_A1,time_bin_A2,time_bin_A3,time_bin_A4,time_bin_M1,time_bin_M2,time_bin_M3,time_bin_M4
0,000004.SZ,2023-08-21,1.632163,8.968361,0.000005,0.294682,4.610838e+05,4.027191e+05,0.753167,0.657830,0.054933,0.059586,0.080318,0.125032,0.493800,0.079928,0.068241,0.038163
1,000004.SZ,2023-08-22,1.547498,8.702269,0.000004,0.491875,5.907764e+05,4.430421e+05,0.800026,0.599965,0.057941,0.055948,0.237730,0.221722,0.186214,0.131757,0.065218,0.043470
2,000004.SZ,2023-08-23,4.197470,45.139436,0.000015,0.171985,1.180099e+06,8.753989e+05,0.803150,0.595777,0.048471,0.038868,0.068469,0.152740,0.154283,0.389039,0.102658,0.045471
3,000004.SZ,2023-08-24,1.445023,7.397598,0.000010,0.447958,6.936484e+05,6.053671e+05,0.753424,0.657535,0.072351,0.033978,0.054979,0.159800,0.425873,0.085955,0.125682,0.041382
4,000004.SZ,2023-08-25,-1.297136,23.930925,0.000012,0.503549,7.222070e+05,6.977789e+05,0.719165,0.694840,0.050146,0.066213,0.042563,0.090724,0.355180,0.192235,0.144538,0.058401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10589,603982.SH,2023-10-26,2.373152,25.816678,0.000012,0.303685,7.915792e+05,7.065599e+05,0.746035,0.665907,0.286909,0.153276,0.095786,0.129383,0.170760,0.045489,0.054065,0.064330
10590,603982.SH,2023-10-27,1.253776,8.945336,0.000007,0.373976,8.088329e+05,6.834076e+05,0.763847,0.645398,0.059306,0.058228,0.067296,0.127246,0.455119,0.134764,0.068868,0.029174
10591,603982.SH,2023-10-30,-0.102672,11.926577,0.000007,-0.075088,5.508855e+05,5.361028e+05,0.716657,0.697426,0.050516,0.102709,0.046097,0.113764,0.479764,0.107870,0.061935,0.037345
10592,603982.SH,2023-10-31,-0.345534,5.892632,0.000003,0.348346,4.777593e+05,4.769197e+05,0.707728,0.706485,0.056225,0.065717,0.086545,0.156295,0.293767,0.163132,0.121280,0.057039


In [11]:
non_hf_factors=pd.read_csv('non-hf-factors.csv')

non_hf_factors['time'] = pd.to_datetime(non_hf_factors['time'])
non_hf_factors['date'] = non_hf_factors['time'].dt.date

non_hf_factors=non_hf_factors.iloc[0:11049,:]
non_hf_factors

# 合并到result 中
new_result = pd.merge(non_hf_factors,result , on=['thscode', 'date'], how='left')
new_result['ths_chg_ratio_stock_shifted'] = new_result['ths_chg_ratio_stock'].shift(-1)
new_result.loc[new_result['time'] == '2023-11-03', 'ths_chg_ratio_stock'] = np.nan
new_result


,time,thscode,ths_stock_short_name_stock,ths_controlling_holder_held_ratio_stock,ths_the_sw_industry_index_code_stock,ths_bod_num_stock,ths_top10_hlolder_held_ratio_stock,ths_float_holder_held_ratio_stock,ths_accounts_payable_pit_stock,ths_operating_total_revenue_pit_stock,...,volume_downward_volatility_ratio,time_bin_A1,time_bin_A2,time_bin_A3,time_bin_A4,time_bin_M1,time_bin_M2,time_bin_M3,time_bin_M4,ths_chg_ratio_stock_shifted
0,2023-08-21,000004.SZ,国华网安,18.22,801750.0,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,...,0.657830,0.054933,0.059586,0.080318,0.125032,0.493800,0.079928,0.068241,0.038163,3.277712
1,2023-08-22,000004.SZ,国华网安,18.22,801750.0,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,...,0.599965,0.057941,0.055948,0.237730,0.221722,0.186214,0.131757,0.065218,0.043470,0.980958
2,2023-08-23,000004.SZ,国华网安,18.22,801750.0,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,...,0.595777,0.048471,0.038868,0.068469,0.152740,0.154283,0.389039,0.102658,0.045471,-3.028571
3,2023-08-24,000004.SZ,国华网安,18.22,801750.0,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,...,0.657535,0.072351,0.033978,0.054979,0.159800,0.425873,0.085955,0.125682,0.041382,-8.662345
4,2023-08-25,000004.SZ,国华网安,18.22,801750.0,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,...,0.694840,0.050146,0.066213,0.042563,0.090724,0.355180,0.192235,0.144538,0.058401,-1.419355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023-10-30,603982.SH,泉峰汽车,27.55,801880.0,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,...,0.697426,0.050516,0.102709,0.046097,0.113764,0.479764,0.107870,0.061935,0.037345,-2.508039
11045,2023-10-31,603982.SH,泉峰汽车,27.55,801880.0,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,...,0.706485,0.056225,0.065717,0.086545,0.156295,0.293767,0.163132,0.121280,0.057039,10.026385
11046,2023-11-01,603982.SH,泉峰汽车,27.55,801880.0,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,...,0.683122,0.039091,0.008595,0.004745,0.012716,0.138445,0.415431,0.353631,0.027346,-0.779376
11047,2023-11-02,603982.SH,泉峰汽车,27.55,801880.0,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.604230


In [209]:

columns_to_lag = [
    'ths_trans_num_stock', 'ths_vaild_turnover_stock', 'ths_vol_btin_stock',
    'ths_amt_btin_stock', 'daily_skewness', 'daily_kurtosis', 'daily_variance',
    'pearson_corr', 'volume_upward_volatility', 'volume_downward_volatility',
    'volume_upward_volatility_ratio', 'volume_downward_volatility_ratio',
    'time_bin_A1', 'time_bin_A2', 'time_bin_A3', 'time_bin_A4',
    'time_bin_M1', 'time_bin_M2', 'time_bin_M3', 'time_bin_M4',
    'ths_chg_ratio_stock_shifted'
]

# 对于每个指定的列，创建昨天和前天的数据列
for column in columns_to_lag:
    new_result[column + '_lag1'] = new_result.groupby('thscode')[column].shift(1)
    new_result[column + '_lag2'] = new_result.groupby('thscode')[column].shift(2)
    new_result[column + '_lag3'] = new_result.groupby('thscode')[column].shift(3)



new_result.to_csv('new_result.csv',index=False)
new_result


,time,thscode,ths_stock_short_name_stock,ths_controlling_holder_held_ratio_stock,ths_the_sw_industry_index_code_stock,ths_bod_num_stock,ths_top10_hlolder_held_ratio_stock,ths_float_holder_held_ratio_stock,ths_accounts_payable_pit_stock,ths_operating_total_revenue_pit_stock,...,volume_downward_volatility_ratio_lag3,time_bin_A1_lag3,time_bin_A2_lag3,time_bin_A3_lag3,time_bin_A4_lag3,time_bin_M1_lag3,time_bin_M2_lag3,time_bin_M3_lag3,time_bin_M4_lag3,ths_chg_ratio_stock_shifted_lag3
0,2023-08-21,000004.SZ,国华网安,18.22,801750.0,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-22,000004.SZ,国华网安,18.22,801750.0,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-08-23,000004.SZ,国华网安,18.22,801750.0,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-24,000004.SZ,国华网安,18.22,801750.0,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,...,0.657830,0.054933,0.059586,0.080318,0.125032,0.493800,0.079928,0.068241,0.038163,3.277712
4,2023-08-25,000004.SZ,国华网安,18.22,801750.0,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,...,0.599965,0.057941,0.055948,0.237730,0.221722,0.186214,0.131757,0.065218,0.043470,0.980958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023-10-30,603982.SH,泉峰汽车,27.55,801880.0,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,...,0.643910,0.073766,0.039628,0.053967,0.115167,0.496688,0.099897,0.068442,0.052447,4.506296
11045,2023-10-31,603982.SH,泉峰汽车,27.55,801880.0,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,...,0.665907,0.286909,0.153276,0.095786,0.129383,0.170760,0.045489,0.054065,0.064330,0.570704
11046,2023-11-01,603982.SH,泉峰汽车,27.55,801880.0,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,...,0.645398,0.059306,0.058228,0.067296,0.127246,0.455119,0.134764,0.068868,0.029174,-1.954603
11047,2023-11-02,603982.SH,泉峰汽车,27.55,801880.0,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,...,0.697426,0.050516,0.102709,0.046097,0.113764,0.479764,0.107870,0.061935,0.037345,-2.508039


In [210]:

data=new_result[new_result['time']!='2023-11-02']
data=data[data['time']!='2023-11-03']
data=data[data['time']!='2023-08-22']
data=data[data['time']!='2023-08-21']
data=data[data['time']!='2023-08-23']

condition1 = (data['ths_stock_short_name_stock'] == '金圆股份')
condition2 = (data['ths_stock_short_name_stock'] == '高新发展') & (data['time'].between('2023-09-27', '2023-10-18'))
condition3 = (data['ths_stock_short_name_stock'] == '创新医疗') & (data['time'].between('2023-08-21', '2023-09-18'))

# 应用条件并去除这些行
data = data[~(condition1 | condition2 | condition3)]

data=data.drop(['time','thscode','ths_stock_short_name_stock','date','ths_controlling_holder_held_ratio_stock','ths_the_sw_industry_index_code_stock'],axis=1)
data=data.reset_index(drop=True)
data

,ths_bod_num_stock,ths_top10_hlolder_held_ratio_stock,ths_float_holder_held_ratio_stock,ths_accounts_payable_pit_stock,ths_operating_total_revenue_pit_stock,ths_eps_basic_stock,ths_nav_ps_stock,ths_oips_stock,ths_pe_lyr_stock,ths_pb_mrq_stock,...,volume_downward_volatility_ratio_lag3,time_bin_A1_lag3,time_bin_A2_lag3,time_bin_A3_lag3,time_bin_A4_lag3,time_bin_M1_lag3,time_bin_M2_lag3,time_bin_M3_lag3,time_bin_M4_lag3,ths_chg_ratio_stock_shifted_lag3
0,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,-0.2652,2.3969,0.6085,-3.774587,6.921337,...,0.657830,0.054933,0.059586,0.080318,0.125032,0.493800,0.079928,0.068241,0.038163,3.277712
1,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,-0.2652,2.3969,0.6085,-3.447619,6.321787,...,0.599965,0.057941,0.055948,0.237730,0.221722,0.186214,0.131757,0.065218,0.043470,0.980958
2,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,-0.2652,2.3969,0.6085,-3.398685,6.232058,...,0.595777,0.048471,0.038868,0.068469,0.152740,0.154283,0.389039,0.102658,0.045471,-3.028571
3,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,-0.2652,2.3969,0.6085,-3.527693,6.468616,...,0.657535,0.072351,0.033978,0.054979,0.159800,0.425873,0.085955,0.125682,0.041382,-8.662345
4,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,-0.2652,2.3969,0.6085,-3.676719,6.741880,...,0.694840,0.050146,0.066213,0.042563,0.090724,0.355180,0.192235,0.144538,0.058401,-1.419355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9845,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,-1.4390,8.8971,5.7672,-26.705060,1.660733,...,0.630304,0.062024,0.067688,0.121715,0.178645,0.272955,0.130793,0.105556,0.060624,-0.810811
9846,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,-1.4390,8.8971,5.7672,-26.857467,1.670211,...,0.698356,0.053686,0.041247,0.054498,0.095997,0.465790,0.183176,0.082131,0.023475,2.792916
9847,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,-1.4390,8.8971,5.7672,-26.332510,1.747769,...,0.643910,0.073766,0.039628,0.053967,0.115167,0.496688,0.099897,0.068442,0.052447,4.506296
9848,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,-1.4390,8.8971,5.7672,-25.672080,1.703934,...,0.665907,0.286909,0.153276,0.095786,0.129383,0.170760,0.045489,0.054065,0.064330,0.570704


In [212]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X=data.drop('ths_chg_ratio_stock_shifted',axis=1)
y=data['ths_chg_ratio_stock_shifted']

# 拆分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

# 转换数据为 DMatrix 格式，这是 XGBoost 的优化数据结构
dtrain = xgb.DMatrix(X_train, label=y_train,enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test,enable_categorical=True)

# 设置训练参数
params = {
    'max_depth': 10,              # 树的最大深度
    'eta': 0.01,                  # 学习率
    'objective': 'reg:squarederror', # 回归任务的目标函数
    'eval_metric': 'rmse'# 评估指标为均方根误差
}

# 训练模型
bst = xgb.train(params, dtrain, num_boost_round=150)

# 预测测试集
preds = bst.predict(dtest)
print(preds)

# 计算均方根误差
rmse = mean_squared_error(y_test, preds, squared=False)
print(f'RMSE: {rmse:.2f}')

[0.370378   0.09484829 0.2914089  ... 0.7022465  0.2794938  0.22013953]
RMSE: 3.36


In [ ]:
from sklearn.model_selection import GridSearchCV

X=data.drop('ths_chg_ratio_stock_shifted',axis=1)
y=data['ths_chg_ratio_stock_shifted']

# 拆分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)


xgbr = xgb.XGBRegressor(objective='reg:squarederror')

params = {
    'max_depth': [10],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'gamma': [0.1],
    'n_estimators': [100, 200],
    'reg_lambda': [1, 1.5],
    'reg_alpha': [0.1]
}

grid_search = GridSearchCV(estimator=xgbr, param_grid=params, cv=2, n_jobs=-1, verbose=1, scoring='neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)

# 输出最佳参数
print(grid_search.best_params_)

# 使用最佳参数重新训练模型
bst = xgb.train(grid_search.best_params_, dtrain, num_boost_round=100)

# 预测测试集
preds = bst.predict(dtest)

# 计算均方根误差
rmse = mean_squared_error(y_test, preds, squared=False)
print(f'Optimized RMSE: {rmse:.2f}')

In [213]:
# 选出 preds 最大的 30 个值的索引
top_30_idx = np.argsort(preds)[-20:]

# 对应的真实值和预测值
top_30_true = y_test.iloc[top_30_idx]
top_30_preds = preds[top_30_idx]

a=top_30_true.mean()

# 计算正负号预测的准确率
top_30_signs_match = np.sign(top_30_true) == np.sign(top_30_preds)
top_30_trend_accuracy = top_30_signs_match.mean()

print(top_30_trend_accuracy)
print(top_30_true,top_30_preds)

0.7
4807    -7.741059
4912    10.004018
7073     3.137255
520      2.199224
7519    -9.995436
4399    -1.944012
8058   -10.014728
7495     3.753213
9784    -1.546961
8357    10.033167
8057     9.989201
3075    10.000000
5335     6.118287
8361    10.016978
4088   -10.023866
4013    10.017889
602      9.993715
9697     0.068587
600     10.000000
9449    10.000000
Name: ths_chg_ratio_stock_shifted, dtype: float64 [3.528051  3.6315563 3.9276483 3.9839087 4.004397  4.0845294 4.1121054
 4.171132  4.1740127 4.1979017 4.5818257 5.06587   5.773094  6.259311
 6.5884786 6.791056  7.1050334 7.691284  7.7782626 7.7782626]


In [219]:

data=new_result[new_result['time']!='2023-11-02']
data=data[data['time']!='2023-11-03']
data=data[data['time']!='2023-08-22']
data=data[data['time']!='2023-08-21']
data=data[data['time']!='2023-08-21']

condition1 = (data['ths_stock_short_name_stock'] == '金圆股份')
condition2 = (data['ths_stock_short_name_stock'] == '高新发展') & (data['time'].between('2023-09-27', '2023-10-18'))
condition3 = (data['ths_stock_short_name_stock'] == '创新医疗') & (data['time'].between('2023-08-21', '2023-09-18'))

# 应用条件并去除这些行
data = data[~(condition1 | condition2 | condition3)]

#data=data.drop(['thscode','ths_stock_short_name_stock','date','ths_controlling_holder_held_ratio_stock','ths_the_sw_industry_index_code_stock'],axis=1)
data=data.drop(['date','ths_controlling_holder_held_ratio_stock','ths_the_sw_industry_index_code_stock'],axis=1)
data=data.reset_index(drop=True)
data=data.dropna()
data

,time,thscode,ths_stock_short_name_stock,ths_bod_num_stock,ths_top10_hlolder_held_ratio_stock,ths_float_holder_held_ratio_stock,ths_accounts_payable_pit_stock,ths_operating_total_revenue_pit_stock,ths_eps_basic_stock,ths_nav_ps_stock,...,volume_downward_volatility_ratio_lag3,time_bin_A1_lag3,time_bin_A2_lag3,time_bin_A3_lag3,time_bin_A4_lag3,time_bin_M1_lag3,time_bin_M2_lag3,time_bin_M3_lag3,time_bin_M4_lag3,ths_chg_ratio_stock_shifted_lag3
1,2023-08-24,000004.SZ,国华网安,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,-0.2652,2.3969,...,0.657830,0.054933,0.059586,0.080318,0.125032,0.493800,0.079928,0.068241,0.038163,3.277712
2,2023-08-25,000004.SZ,国华网安,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,-0.2652,2.3969,...,0.599965,0.057941,0.055948,0.237730,0.221722,0.186214,0.131757,0.065218,0.043470,0.980958
3,2023-08-28,000004.SZ,国华网安,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,-0.2652,2.3969,...,0.595777,0.048471,0.038868,0.068469,0.152740,0.154283,0.389039,0.102658,0.045471,-3.028571
4,2023-08-29,000004.SZ,国华网安,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,-0.2652,2.3969,...,0.657535,0.072351,0.033978,0.054979,0.159800,0.425873,0.085955,0.125682,0.041382,-8.662345
5,2023-08-30,000004.SZ,国华网安,5.0,38.26,18.2200,9.097724e+07,2.125577e+07,-0.2652,2.3969,...,0.694840,0.050146,0.066213,0.042563,0.090724,0.355180,0.192235,0.144538,0.058401,-1.419355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10068,2023-10-26,603982.SH,泉峰汽车,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,-1.4390,8.8971,...,0.630304,0.062024,0.067688,0.121715,0.178645,0.272955,0.130793,0.105556,0.060624,-0.810811
10069,2023-10-27,603982.SH,泉峰汽车,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,-1.4390,8.8971,...,0.698356,0.053686,0.041247,0.054498,0.095997,0.465790,0.183176,0.082131,0.023475,2.792916
10070,2023-10-30,603982.SH,泉峰汽车,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,-1.4390,8.8971,...,0.643910,0.073766,0.039628,0.053967,0.115167,0.496688,0.099897,0.068442,0.052447,4.506296
10071,2023-10-31,603982.SH,泉峰汽车,7.0,55.73,27.5472,4.137170e+08,4.463180e+08,-1.4390,8.8971,...,0.665907,0.286909,0.153276,0.095786,0.129383,0.170760,0.045489,0.054065,0.064330,0.570704


In [220]:
data=data.dropna()
data.to_csv('data.csv')

In [215]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X=data.drop('ths_chg_ratio_stock_shifted',axis=1)
y=data[['ths_chg_ratio_stock_shifted','time']]

# 拆分数据集为训练集和测试集
X_train=X[X['time']!='2023-10-17']
y_train=y[y['time']!='2023-10-17']
X_test=X[X['time']=='2023-10-17']
y_test=y[y['time']=='2023-10-17']

X_train=X_train.drop(['time'],axis=1)
y_train=y_train.drop('time',axis=1)
X_test=X_test.drop(['time'],axis=1)
y_test=y_test.drop('time',axis=1)

y_train = y_train['ths_chg_ratio_stock_shifted']
y_test = y_test['ths_chg_ratio_stock_shifted']

# 转换数据为 DMatrix 格式，这是 XGBoost 的优化数据结构
dtrain = xgb.DMatrix(X_train, label=y_train,enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test,enable_categorical=True)

# 设置训练参数
params = {
    'max_depth': 10,              # 树的最大深度
    'eta': 0.01,                  # 学习率
    'objective': 'reg:squarederror', # 回归任务的目标函数
    'eval_metric': 'rmse'# 评估指标为均方根误差
}

# 训练模型
bst = xgb.train(params, dtrain, num_boost_round=100)

# 预测测试集
preds = bst.predict(dtest)
print(preds)

# 计算均方根误差
rmse = mean_squared_error(y_test, preds, squared=False)
print(f'RMSE: {rmse:.2f}')

[-1.24811873e-01  1.51337042e-01  3.79275411e-01  3.98150623e-01
  6.78457879e-03  4.25466970e-02  3.63048971e-01  1.19524404e-01
 -2.56264091e-01  3.95539820e-01  3.71079206e-01  7.81309843e-01
  5.39136052e-01  5.08902431e-01  6.39064729e-01  8.89397711e-02
  8.40098917e-01  3.76384318e-01 -2.59989016e-02  1.32240683e-01
 -1.95291162e-01  6.88897848e-01  3.00779879e-01 -1.06776841e-01
  4.23945099e-01  3.49990278e-01  5.78634977e-01  7.96044111e-01
  1.91644382e+00  7.54927933e-01  2.23272219e-01  3.92604560e-01
  7.51507878e-02  2.73267359e-01  4.47795063e-01  4.23960447e-01
  1.03851914e-01  1.03927517e+00  2.01863050e+00  6.24698281e-01
 -3.01812917e-01  5.47747016e-01  5.56573868e-01  1.21603690e-01
 -1.59415781e-01  2.06433833e-01  6.69906497e-01  1.75896101e-02
  2.01273903e-01  3.26873839e-01  3.86427402e-01  4.13139850e-01
 -8.69897604e-01  6.46935701e-01  3.68940830e-01  3.77933413e-01
  4.50728029e-01 -9.58341241e-01  5.50107300e-01  5.34944475e-01
 -7.87004769e-01  3.96822

In [218]:
# 选出 preds 最大的 30 个值的索引
top_30_idx = np.argsort(preds)[-10:]

# 对应的真实值和预测值
top_30_true = y_test.iloc[top_30_idx]
top_30_preds = preds[top_30_idx]

a=top_30_true.mean()

# 计算正负号预测的准确率
top_30_signs_match = np.sign(top_30_true) == np.sign(top_30_preds)
top_30_trend_accuracy = top_30_signs_match.mean()

print(a)
print(top_30_trend_accuracy)
print(top_30_true,top_30_preds)

2.2512224760999997
0.7
4009    -1.237624
4054    -0.904466
8734    -6.103286
5539     2.405180
9431     2.529762
1373     9.993338
1823     2.759949
7339     0.816743
4369    10.005438
5674     2.247191
Name: ths_chg_ratio_stock_shifted, dtype: float64 [1.4571978 1.489827  1.5788171 1.6086912 1.8397615 1.9164438 2.0186305
 2.2160368 3.1042476 4.0218263]
